In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
model = UserKNNCFRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2590 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    topK = trial.suggest_int("topK", 1e2, 1e4)
    shrink = trial.suggest_int("shrink", 0, 100)
    similarity = trial.suggest_categorical("similarity", ["cosine", "jaccard", "asymmetric", "dice", "tversky"])
    normalize = trial.suggest_categorical("normalize", [True, False])
    feature_weighting = trial.suggest_categorical("feature_weighting", ["none", "BM25", "TF-IDF"])

    
    recommender = model(URM_train)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [4]:
best_params = {
    'topK': 529,
    'shrink': 45,
    'similarity': 'asymmetric',
    'normalize': True,
    'feature_weighting': 'TF-IDF'
}
study = op.create_study(direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-11-22 00:36:28,404] A new study created in memory with name: no-name-7fd53cc8-f7ec-48dc-b04b-28654f52c578


UserKNNCFRecommender: URM Detected 484 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 319 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 8695.59 column/sec. Elapsed time 1.45 sec
EvaluatorHoldout: Processed 10048 (100.0%) in 6.86 sec. Users per second: 1465


[I 2023-11-22 00:36:36,876] Trial 0 finished with value: 0.023100230007077137 and parameters: {'topK': 529, 'shrink': 45, 'similarity': 'asymmetric', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023100230007077137.


UserKNNCFRecommender: URM Detected 484 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 319 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 9431.13 column/sec. Elapsed time 1.34 sec
EvaluatorHoldout: Processed 10048 (100.0%) in 11.00 sec. Users per second: 914


[I 2023-11-22 00:36:49,772] Trial 1 finished with value: 0.014974068913658845 and parameters: {'topK': 7188, 'shrink': 83, 'similarity': 'asymmetric', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023100230007077137.


...

[I 2023-11-22 00:54:42,093] Trial 97 finished with value: 0.02193426315589934 and parameters: {'topK': 380, 'shrink': 17, 'similarity': 'tversky', 'normalize': False, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023100230007077137.


UserKNNCFRecommender: URM Detected 484 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 319 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 10090.90 column/sec. Elapsed time 1.25 sec
EvaluatorHoldout: Processed 10048 (100.0%) in 6.10 sec. Users per second: 1646


[I 2023-11-22 00:54:49,609] Trial 98 finished with value: 0.021875699025629355 and parameters: {'topK': 431, 'shrink': 20, 'similarity': 'tversky', 'normalize': False, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023100230007077137.


UserKNNCFRecommender: URM Detected 484 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 319 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 10098.55 column/sec. Elapsed time 1.25 sec
EvaluatorHoldout: Processed 10048 (100.0%) in 6.10 sec. Users per second: 1646


[I 2023-11-22 00:54:57,125] Trial 99 finished with value: 0.021965715359923168 and parameters: {'topK': 425, 'shrink': 12, 'similarity': 'tversky', 'normalize': False, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023100230007077137.


In [5]:
study.best_params

{'topK': 529,
 'shrink': 45,
 'similarity': 'asymmetric',
 'normalize': True,
 'feature_weighting': 'TF-IDF'}

In [7]:
final = model(URM_train_validation)
final.fit(**study.best_params)

SLIMElasticNetRecommender: URM Detected 232 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 103 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 17389 (78.3%) in 5.00 min. Items per second: 57.96
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 6.45 min. Items per second: 57.42


In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2176 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10462 (100.0%) in 6.56 sec. Users per second: 1596


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.096683                 0.167108  0.140668  0.049947    0.086206   
 
             MRR      NDCG      F1  HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                     ...                 
 10      0.27832  0.154074  0.1146  0.527337      0.368245  ...      0.827821   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.436541    0.827821       0.054661        10.16803   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.997643             0.156901              0.783344   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff